# Load the data


In [ ]:
https://www.kaggle.com/code/liamhealy/lightgbm-hyperparameter-tuning

In [ ]:
https://github.com/BlakSwan/Home-Credit/blob/main/home-credit-starter-eda-and-submission.ipynb
    
https://www.kaggle.com/code/benjenkins96/deep-learning-techniques-for-credit-risk-stability

In [1]:
!pip install polars
!pip install lightgbm
#!pip install skopt

In [2]:
import pandas as pd
import polars as pl
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
#import skopt
import sys
import os

pd.set_option('display.max_columns', None)

#dataPath = "/kaggle/input/home-credit-credit-risk-model-stability/data/"
dataPath = "C:/Users/virtualbox/Downloads/kaggle/"

In [3]:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for col in df.columns:
        # last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))
    return df

In [4]:
def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:  
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

In [5]:
def from_polars_to_pandas(case_ids: pl.DataFrame) -> pl.DataFrame:
    return (
        data.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))[cols_pred].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
    )

In [6]:
def summary(df):
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#total'] = df.shape[0]
    summ['#missing'] = df.isnull().sum().values 
    summ['%missing'] = df.isnull().sum().values / len(df)* 100
    summ['#unique'] = df.nunique().values
    summ['#duplicates'] = summ['#total'] - summ['#unique']
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    return summ

In [7]:
def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

def drop_outliers(df, field_name):
    iqr = 1.5 * (np.percentile(df[field_name], 75) - np.percentile(df[field_name], 25))
    df.drop(df[df[field_name] > (iqr + np.percentile(df[field_name], 75))].index, inplace=True)
    df.drop(df[df[field_name] < (np.percentile(df[field_name], 25) - iqr)].index, inplace=True)

MODELLING

In [8]:
keep_vars = list(['target','case_id','date_decision','MONTH','WEEK_NUM','price_1097A',
'mobilephncnt_593L',
'pmtnum_254L',
'avgdpdtolclosure24_3658938P',
'numrejects9m_859L',
'cntpmts24_3658933L',
'numinstunpaidmax_3546851L',
'eir_270L',
'numinstlsallpaid_934L',
'lastdelinqdate_224D',
'maxdbddpdtollast12m_3658940P',
'numincomingpmts_3546848L',
'pctinstlsallpaidlate1d_3546856L',
'maxdpdlast3m_392P',
'monthsannuity_845L',
'datelastinstal40dpd_247D',
'lastrejectreason_759M',
'pmtssum_45A',
'education_1103M',
'riskassesment_940T',
'pmtaverage_3A',
'days30_165L',
'days360_512L',
'days180_256L',
'thirdquarter_1082L',
'riskassesment_302T',
'days120_123L',
'firstquarter_103L',
'registaddr_district_1083M',
'education_927M',
'mainoccupationinc_384A',
'incometype_1044T',
'empladdr_district_926M',
'housetype_905L',
'type_25L',
'amount_416A',
'last180dayturnover_1134A',
'last180dayaveragebalance_704A',
'last30dayturnover_651A',
'dateofcredstart_739D',
'residualamount_856A',
'refreshdate_3813885D',
'dateofcredstart_181D',
'numberofoverdueinstlmax_1039L',
'totalamount_996A',
'dpdmaxdateyear_596T',
'dpdmax_757P',
'description_351M',
'totaldebtoverduevalue_178A',
'dpdmax_139P',
'totalamount_6A',
'lastupdate_1112D',
'overdueamountmax_35A',
'overdueamountmax2_398A',
'totaloutstanddebtvalue_39A',
'numberofinstls_320L',
'numberofoverdueinstlmaxdat_148D',
'overdueamountmax2_14A',
'numberofcontrsvalue_258L'])


# TRAINING DATA SET

In [ ]:
### BASE TABLE
train_basetable = pl.read_csv(dataPath + "train/train_base.csv")

### STATICS
train_static = pl.concat(
    [
        pl.read_csv(dataPath + "train/train_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "train/train_static_0_1.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)

In [ ]:
### STATIC CB
train_static_cb = pl.read_csv(dataPath + "train/train_static_cb_0.csv").pipe(set_table_dtypes)

### PERSON 1
train_person_1 = pl.read_csv(dataPath + "train/train_person_1.csv").filter((pl.col("num_group1") == 0)).drop("num_group1").pipe(set_table_dtypes) 


In [ ]:
### DEPOSIT 1
train_deposit_1 = pl.read_csv(dataPath + "train/train_deposit_1.csv").filter((pl.col("num_group1") == 0)).drop("num_group1").pipe(set_table_dtypes) 

### DEBIT CARD 1
train_debitcard_1 = pl.read_csv(dataPath + "train/train_debitcard_1.csv").filter((pl.col("num_group1") == 0)).drop("num_group1").pipe(set_table_dtypes) 



In [ ]:
### APP PREV 1
train_applprev_1 = pl.concat(
    [
        pl.read_csv(dataPath + "train/train_applprev_1_0.csv").filter((pl.col("num_group1") == 0)).drop("num_group1").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "train/train_applprev_1_1.csv").filter((pl.col("num_group1") == 0)).drop("num_group1").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)

### OTHER 1
train_other_1 = pl.read_csv(dataPath + "train/train_other_1.csv").filter((pl.col("num_group1") == 0)).drop("num_group1").pipe(set_table_dtypes) 

In [31]:
### CREDIT BUREAU A1
train_credit_bureau_a_1 = pl.concat(
    [
        pl.read_csv(dataPath + "train/train_credit_bureau_a_1_0.csv").filter((pl.col("num_group1") == 0)).drop("num_group1").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "train/train_credit_bureau_a_1_1.csv").filter((pl.col("num_group1") == 0)).drop("num_group1").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "train/train_credit_bureau_a_1_2.csv").filter((pl.col("num_group1") == 0)).drop("num_group1").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "train/train_credit_bureau_a_1_3.csv").filter((pl.col("num_group1") == 0)).drop("num_group1").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)


shape: (1_526_659, 5)
┌─────────┬───────────────┬────────┬──────────┬────────┐
│ case_id ┆ date_decision ┆ MONTH  ┆ WEEK_NUM ┆ target │
│ ---     ┆ ---           ┆ ---    ┆ ---      ┆ ---    │
│ i64     ┆ str           ┆ i64    ┆ i64      ┆ i64    │
╞═════════╪═══════════════╪════════╪══════════╪════════╡
│ 0       ┆ 2019-01-03    ┆ 201901 ┆ 0        ┆ 0      │
│ 1       ┆ 2019-01-03    ┆ 201901 ┆ 0        ┆ 0      │
│ 2       ┆ 2019-01-04    ┆ 201901 ┆ 0        ┆ 0      │
│ 3       ┆ 2019-01-03    ┆ 201901 ┆ 0        ┆ 0      │
│ 4       ┆ 2019-01-04    ┆ 201901 ┆ 0        ┆ 1      │
│ …       ┆ …             ┆ …      ┆ …        ┆ …      │
│ 2703450 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
│ 2703451 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
│ 2703452 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
│ 2703453 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
│ 2703454 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
└─────────┴───────────────┴────────┴──────────┴────────┘


In [ ]:
### CREDIT BUREAU B1
train_credit_bureau_b_1 = pl.read_csv(dataPath + "csv_files/train/train_credit_bureau_b_1.csv").filter((pl.col("num_group1") == 0)).drop("num_group1").pipe(set_table_dtypes)

### TAX REGISTERY A1
train_tax_registry_a_1 = pl.read_csv(dataPath + "csv_files/train/train_tax_registry_a_1.csv").filter((pl.col("num_group1") == 0)).drop("num_group1").pipe(set_table_dtypes)

In [ ]:
# JOIN TABLES TOGETHER
data = train_basetable.join( 
    train_static, how="left", on="case_id").join(
    train_static_cb, how="left", on="case_id").join(
    train_person_1, how="left", on="case_id").join(
    train_deposit_1, how="left", on="case_id").join(
    train_debitcard_1, how="left", on="case_id").join(
    train_applprev_1, how="left", on="case_id").join(
    train_other_1, how="left", on="case_id").join(
    train_credit_bureau_a_1, how="left", on="case_id").join(
    train_credit_bureau_b_1, how="left", on="case_id").join(
    train_tax_registry_a_1, how="left", on="case_id"
).select(keep_vars)

print("Done!")
Done!